In [1]:
import dace

def vec_copy():
    sdfg = dace.SDFG("soft_hier_vec_copy")
    state = sdfg.add_state("main")
    a_host = sdfg.add_array("A", (256*32, ), dace.float16, dace.dtypes.StorageType.CPU_Heap, transient=True)
    a_dev = sdfg.add_array("soft_hier_A", (256*32, ), dace.float16, dace.dtypes.StorageType.SoftHier_HBM, transient=True)
    b_host = sdfg.add_array("B", (256*32, ), dace.float16, dace.dtypes.StorageType.CPU_Heap, transient=True)
    b_dev = sdfg.add_array("soft_hier_B", (256*32, ), dace.float16, dace.dtypes.StorageType.SoftHier_HBM, transient=True)
    ahc = state.add_access("A")
    adc = state.add_access("soft_hier_A")
    bhc = state.add_access("B")
    bdc = state.add_access("soft_hier_B")
    frag_a = sdfg.add_array("frag_A", (512, ), dace.float16, dace.dtypes.StorageType.SoftHier_TCDM, transient=True)
    frag_b = sdfg.add_array("frag_B", (512, ), dace.float16, dace.dtypes.StorageType.SoftHier_TCDM, transient=True)
    #af = state.add_access("frag_A")
    #bf = state.add_access("frag_B")

    dev_entry, dev_exit = state.add_map(name="copy_map_outer", ndrange={"i": dace.subsets.Range([(0, 256*32-1, 8192)])}, schedule=dace.dtypes.ScheduleType.SoftHier_Device)
    tblock_entry, tblock_exit = state.add_map(name="copy_map_inner", ndrange={"ii": dace.subsets.Range([(0, 256*32-1, 512)])}, schedule=dace.dtypes.ScheduleType.SoftHier_Cluster)

    #glb_to_vecin = GlobalToVECIN(name="glb_to_vecin_a", input_names=["IN_A"], output_names=["OUT_frag_A"], queue_length=1, load_length=256)
    glb_to_vecin = state.add_access("frag_A")
    #vecout_to_glb = VECOUTToGlobal(name="vecout_to_glb_b", input_names=["IN_frag_B"], output_names=["OUT_B"], queue_length=1, load_length=256)
    libnode = state.add_access("frag_B")
    #libnode = VecUnit(name="vecout_to_glb_b", input_names=["IN_frag_A"], output_names=["OUT_frag_B"], queue_length=1, load_length=256)

    state.add_edge(ahc, None, adc, None, dace.memlet.Memlet(f"A[0:{256*32}]"))
    state.add_edge(adc, None, dev_entry, "IN_A", dace.memlet.Memlet(f"soft_hier_A[0:{256*32}]"))
    state.add_edge(dev_entry, "OUT_A", tblock_entry, "IN_A", dace.memlet.Memlet(f"soft_hier_A[0:{256*32}]"))
    state.add_edge(tblock_entry, "OUT_A", glb_to_vecin, None, dace.memlet.Memlet(f"soft_hier_A[i + ii:i + ii + 512]"))
    state.add_edge(glb_to_vecin, None, libnode, None, dace.memlet.Memlet(f"frag_A[0:512]"))
    state.add_edge(libnode, None, tblock_exit, "IN_B", dace.memlet.Memlet(f"soft_hier_B[i + ii:i + ii + 512]"))
    state.add_edge(tblock_exit, "OUT_B", dev_exit, "IN_B", dace.memlet.Memlet(f"soft_hier_B[i + ii:i + ii + 512]"))
    state.add_edge(dev_exit, "OUT_B", bdc, None, dace.memlet.Memlet(f"soft_hier_B[0:{256*32}]"))
    state.add_edge(bdc, None, bhc, None, dace.memlet.Memlet(f"B[0:{256*32}]"))

    for n in [dev_entry, tblock_entry]:
        n.add_in_connector("IN_A")
        n.add_out_connector("OUT_A")

    for n in [dev_exit, tblock_exit]:
        n.add_in_connector("IN_B")
        n.add_out_connector("OUT_B")

    #libnode.add_in_connector("IN_A")
    #libnode.add_out_connector("OUT_B")

    #t = state.add_tasklet(name="assign", inputes={"_in"}, outputs={"_out"}, code="_out = _in")


    sdfg.save("soft_hier_2.sdfgz")
    return sdfg

In [2]:
s = vec_copy()

In [3]:
from IPython.display import Code
# Code(s.generate_code()[0].clean_code, language='cpp')

In [4]:
Code(s.generate_code()[1].clean_code, language='cpp')

Using frame code generator for soft_hier_vec_copy
Allocated outer-level transients
Generating states
Generating states for SDFG soft_hier_vec_copy...
Using control flow detection...
Structured control flow tree
Reset block parents
Generating code for cft...
class GeneralBlock as_cpp
length of elements: 1
generate_transition: 0 for type <class 'dace.codegen.control_flow.BasicCFBlock'>
class BasicCFBlock as_cpp
state main has 10 nodes
Calling dispatch_state main...
Dispatching state main with DaCeCodeGenerator
Framecode generating state main...
Allocating arrays in scope for state main...
SoftHier: Allocating soft_hier_A
SoftHier: Allocating soft_hier_B
Finished allocating arrays in scope for state main.
len(components) <= 1: 1
~~~~~~~~~~~~~~~Dispatching subgraph main in state 0
***************Dispatching node A in state 0
11111111111
CPU: Generating node A in state 0
SoftHier: Emitting copy from A to soft_hier_A
  main 0 A soft_hier_A A:None  -(A[0:8192])->  soft_hier_A:None None <dace.

/scratch/dace4softhier/dace_soft_hier/dace/sdfg/validation.py:519: UserWarning: WARNING: Use of uninitialized transient "A" in state "main"
  warnings.warn('WARNING: Use of uninitialized transient "%s" in state "%s"' %
/scratch/dace4softhier/dace_soft_hier/dace/codegen/targets/soft_hier/soft_hier.py:1803: UserWarning: No `gpu_block_size` property specified on map "copy_map_outer". Falling back to the configuration entry `compiler.cuda.default_block_size`: 32,1,1. You can either specify the block size to use with the gpu_block_size property, or by adding nested `GPU_ThreadBlock` maps, which map work to individual threads. For more information, see https://spcldace.readthedocs.io/en/latest/optimization/gpu.html
  warnings.warn(


#include <cuda_runtime.h>
#include <dace/dace.h>


struct soft_hier_vec_copy_state_t {
    int filler;
};



DACE_EXPORTED int __dace_init_cuda(soft_hier_vec_copy_state_t *__state);
DACE_EXPORTED int __dace_exit_cuda(soft_hier_vec_copy_state_t *__state);

#define FRAG_A_SIZE (1024)
#define FRAG_A_ADDR (0)
#define FRAG_B_SIZE (1024)
#define FRAG_B_ADDR ((0 + FRAG_A_SIZE))


int __dace_init_cuda(soft_hier_vec_copy_state_t *__state) {
    
    

    __state->gpu_context = new dace::cuda::Context(1, 1);

    

    return 0;
}

int __dace_exit_cuda(soft_hier_vec_copy_state_t *__state) {
    
    int __err = 0;
    delete __state->gpu_context;
    return __err;
}


void copy_map_outer_0_0_4(uint32_t soft_hier_A, uint32_t soft_hier_B) {
    {
        // TEST KERNEL SCOPE
        flex_global_barrier_xy();
        uint32_t cluster_id = flex_get_cluster_id();
        uint32_t core_id = flex_get_core_id();
        int i = 0;
        flex_global_barrier_xy();
        {
            // TEST DEVICE SCOPE
            uint32_t frag_A = FRAG_A_ADDR;
            uint32_t frag_B = FRAG_B_ADDR;
            int ii = (512 * cluster_id);
            if (ii <= 8191) {
                // Minels: [0], Maxels: [8191]
                // SoftHier_HBM -> SoftHier_TCDM
                if(flex_is_dm_core())
                {
                    flex_dma_async_1d(hbm_addr(soft_hier_A + (i + ii)), local(frag_A), 512*2);
                    flex_dma_async_wait_all();
                }
                flex_intra_cluster_sync();
                // SoftHier_TCDM -> SoftHier_TCDM
                if(flex_is_dm_core())
                {
                    flex_dma_async_1d(local(frag_A), local(frag_B), 512*2);
                    flex_dma_async_wait_all();
                }
                flex_intra_cluster_sync();
                // SoftHier_TCDM -> SoftHier_HBM
                if(flex_is_dm_core())
                {
                    flex_dma_async_1d(local(frag_B), hbm_addr(soft_hier_B + (i + ii)), 512*2);
                    flex_dma_async_wait_all();
                }
                flex_intra_cluster_sync();
            }
        }
        flex_global_barrier_xy();
    }
}


DACE_EXPORTED void __dace_runkernel_copy_map_outer_0_0_4(soft_hier_vec_copy_state_t *__state, uint32_t soft_hier_A, uint32_t soft_hier_B);
void __dace_runkernel_copy_map_outer_0_0_4(soft_hier_vec_copy_state_t *__state, uint32_t soft_hier_A, uint32_t soft_hier_B)
{

    void  *copy_map_outer_0_0_4_args[] = { (void *)&soft_hier_A, (void *)&soft_hier_B };
    gpuError_t __err = cudaLaunchKernel((void*)copy_map_outer_0_0_4, dim3(1, 1, 1), dim3(32, 1, 1), copy_map_outer_0_0_4_args, 0, nullptr);
    DACE_KERNEL_LAUNCH_CHECK(__err, "copy_map_outer_0_0_4", 1, 1, 1, 32, 1, 1);
}